In [7]:
import os
import openai
import langchain as lc
from langchain.document_loaders import SRTLoader
from dotenv import load_dotenv
import srt
import codecs
import tiktoken
from IPython.display import clear_output

# Load environment variables
load_dotenv()

openai.api_key = os.environ.get("OPENAI_API_KEY")

#model_name = "gpt-3.5-turbo-16k"
model_name = "gpt-4"

In [13]:
def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

### Creating video chapters

In [14]:
def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model= model_name,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def process_srt_file(file_path):
    # Read and parse SRT file
    with codecs.open(file_path, 'r', encoding='utf-8-sig') as f:
        file_content = f.read()
        subtitle_generator = srt.parse(file_content)
        subtitles = list(subtitle_generator)
    
    result = ""
    chunk = ""
    for sub in subtitles:
        chunk += str(sub.start) + ">" + str(sub.end.total_seconds) + "\\n" + sub.content
        if num_tokens_from_string(chunk) > 7000:
            prompt = "Below is a part of a video transcript. You need to split the video into five topic chapters. The chapters will be used to navigate in the larger video timeline to let watchers switch between topics. Read the entire transcript. Once done reading, split it into chapters. Provide the list of chapters in this format [HH:MM:SS Chapter Name]. Put each chapter in a separate line in plain text. Match the transcript language in the output.\\n\\n" + chunk
            completion = get_completion(prompt)
            result += completion
            clear_output(wait=True)
            print(completion)
            chunk = ""        

    return result

final_output = process_srt_file("output_audio.srt")
with open('chapters.txt', 'w', encoding='utf-8') as f:
    f.write(final_output)

[0:08:11 Ülke ve Yasaların Etkisi]
[0:08:20 Tazminat ve Bütçe Planlaması]
[0:08:49 Performansa Dayalı Bonuslar ve Takım Bonusları]
[0:09:17 Yan Haklar ve Ücretli İzinler]
[0:09:42 Enflasyon ve Maaş Eşitliği]
[0:14:35 İç ve Dış Çalışan Maaşları Arasındaki Fark]
[0:16:49 İnsan Kaynakları ve Finansın İşbirliği]
[0:17:23 Küçük Start-up'larda Maaş Planlaması]
[0:17:48 Ücret Eşitliği ve Demografik Eşitsizlikler]
[0:18:35 Şirket İçi Maaş Hizalanması ve Finansal Hesaplama Problemleri]


### Creating video summary

In [16]:
def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model= model_name,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def process_srt_file(file_path):
    # Read and parse SRT file
    with codecs.open(file_path, 'r', encoding='utf-8-sig') as f:
        file_content = f.read()
        subtitle_generator = srt.parse(file_content)
        subtitles = list(subtitle_generator)
    
    result = ""
    chunk = ""
    for sub in subtitles:
        chunk += str(sub.start) + ">" + str(sub.end.total_seconds) + "\\n" + sub.content
        if num_tokens_from_string(chunk) > 7000:
            prompt = "Below is a part of a video transcript. Your goal is to summarize the entire video. You need to create the shortest summary of this section as possible that you will combine with other sections to create the full summary of the entire video.\\n\\n" + chunk
            completion = get_completion(prompt)
            result += completion
            clear_output(wait=True)
            print(completion)
            chunk = ""        

    return result

final_output = process_srt_file("output_audio.srt")
with open('summaries.txt', 'w', encoding='utf-8') as f:
    f.write(final_output)

The video discusses the complexities of salary calculations and adjustments within a company. It emphasizes the importance of considering factors such as legal requirements, potential severance payments, performance-based bonuses, and inflation. The speaker also highlights the need for regular reviews of salary equality within the company to prevent disparities between new and existing employees. The video further discusses the concept of a healthy turnover rate and the necessity of aligning internal and external salary evaluations. The speaker concludes by stating that discrepancies in salary adjustments are financial calculation problems, not cultural ones.


In [17]:
def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model= model_name,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def process_srt_file(file_path):
    content = ""
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    result = ""
    prompt = "Below is a series of summaries created out of different section of a video recording. The video is published on Youtube. Provide 10 TUrkish title alternatives and a single Turkish summary for the video. Both title and summary should be inviting and helpful to watchers. \\n\\n" + content
    result = get_completion(prompt)

    return result

final_output = process_srt_file("summaries.txt")
with open('title-description.txt', 'w', encoding='utf-8') as f:
    f.write(final_output)

### Audio download from Youtube

In [3]:

from pytube import YouTube
from pydub import AudioSegment
import subprocess

# YouTube video url
url = 'https://www.youtube.com/watch?v=R--fSB7CBIs'

def download_audio(youtube_link):
    youtube = YouTube(youtube_link)
    audio_stream = youtube.streams.filter(only_audio=True).first()
    # Save as .webm
    audio_stream.download(filename="audio.mp4")

download_audio(url)

### Extract Audio from Video

In [11]:
from moviepy.editor import VideoFileClip

def extract_audio(video_file, audio_file):
    video = VideoFileClip(video_file)
    audio = video.audio
    audio.write_audiofile(audio_file)

# usage
extract_audio('C:\\Users\\daronyondem\\Videos\\2023-09-11-Maaslar-Nasil-Belirleniyor2.mp4', 'output_audio.mp3')

MoviePy - Writing audio in output_audio.mp3


MoviePy - Done.


### Audio Split for Whisper

In [1]:
from pydub import AudioSegment

song = AudioSegment.from_mp3("C:\\github\\dy\\video-transcript-analzyer\\output_audio.mp3")

# PyDub handles time in milliseconds
ten_minutes = 10 * 60 * 1000

first_10_minutes = song[:ten_minutes]

first_10_minutes.export("output_audio_10.mp3", format="mp3")

<_io.BufferedRandom name='output_audio_10.mp3'>

### Whisper Transcription

In [6]:
import whisper
from whisper.utils import get_writer
from pydub import AudioSegment

model = whisper.load_model("medium")
audio = "C:\\github\\dy\\video-transcript-analzyer\\output_audio.mp3"
result = model.transcribe(audio)
output_directory = "./"

# Save as an SRT file
srt_writer = get_writer("srt", output_directory)
srt_writer(result, audio)

100%|█████████████████████████████████████| 1.42G/1.42G [02:04<00:00, 12.2MiB/s]
